<a href="https://colab.research.google.com/github/S18-Niloy/Greatest_players_gpt/blob/main/gpt_dev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [24]:
#read it in
with open("input.txt","r",encoding="utf-8") as f:
  text = f.read()

In [25]:
#length of the text
length = len(text)
print(length)

17354


In [40]:
#checking the first 1000 words
print(text[:1000])

10. George Best

Best’s extraordinary natural talent not only made him one of the most iconic and entertaining players there has been, but the finest from the British Isles.

“If I’d been ugly,” he once said, “You’d never have heard of Pele.” Although that may be a stretch, it was an acknowledgement that, for all of his ability, his womanising stopped him from achieving even more.

It was in 1968 that, aged 22 – his pace, belief, technical ability, balance and creativity in perfect symmetry – Best deservedly followed the other two of United's ‘Holy Trinity’ (Bobby Charlton and Denis Law) in winning the Ballon d’Or.

Best had made his debut for United in 1963 - a boyhood Wolves fan, the Belfast boy was rejected by local club Glentoran for being too small. That didn’t look like a great decision, as the wide man soon began to take British football by storm.

“My best performance was for Northern Ireland against Scotland,” he once told FFT. “We were expected to get slaughtered, but we won 

In [41]:
#all unique  characters
char = sorted(list(set(text)))
len_char = len(char)
print(''.join(char))
print(len_char)


 '(),-./0123456789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyz£ç–‘’“”…
81


In [42]:
#create a mapping from characters to integers
stoi = {ch:i for i, ch in enumerate(char)}
itos = {i:ch for i, ch in enumerate(char)}

encode = lambda s:[stoi[c] for c in s]
decode = lambda l: ''.join([itos[i] for i in l])

x = "messi"
print(encode(x))
print(decode(encode('messi')))

[59, 51, 65, 65, 55]
messi


In [54]:
#encode the entire dataset
import torch
data = torch.tensor(encode(text),dtype = torch.long)
print(data.shape, data.dtype)
print(data[:1000])

torch.Size([17354]) torch.int64
tensor([10,  9,  7,  1, 28, 51, 61, 64, 53, 51,  1, 23, 51, 65, 66,  0,  0, 23,
        51, 65, 66, 77, 65,  1, 51, 70, 66, 64, 47, 61, 64, 50, 55, 60, 47, 64,
        71,  1, 60, 47, 66, 67, 64, 47, 58,  1, 66, 47, 58, 51, 60, 66,  1, 60,
        61, 66,  1, 61, 60, 58, 71,  1, 59, 47, 50, 51,  1, 54, 55, 59,  1, 61,
        60, 51,  1, 61, 52,  1, 66, 54, 51,  1, 59, 61, 65, 66,  1, 55, 49, 61,
        60, 55, 49,  1, 47, 60, 50,  1, 51, 60, 66, 51, 64, 66, 47, 55, 60, 55,
        60, 53,  1, 62, 58, 47, 71, 51, 64, 65,  1, 66, 54, 51, 64, 51,  1, 54,
        47, 65,  1, 48, 51, 51, 60,  5,  1, 48, 67, 66,  1, 66, 54, 51,  1, 52,
        55, 60, 51, 65, 66,  1, 52, 64, 61, 59,  1, 66, 54, 51,  1, 23, 64, 55,
        66, 55, 65, 54,  1, 30, 65, 58, 51, 65,  7,  0,  0, 78, 30, 52,  1, 30,
        77, 50,  1, 48, 51, 51, 60,  1, 67, 53, 58, 71,  5, 79,  1, 54, 51,  1,
        61, 60, 49, 51,  1, 65, 47, 55, 50,  5,  1, 78, 45, 61, 67, 77, 50,  1,
        